In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.5


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [6]:
val sameModel = PipelineModel.load(modelPath)

sameModel = pipeline_68625e8bfb62


pipeline_68625e8bfb62

In [7]:
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [74]:
val predictionsDF = sameModel.transform(inputDF)
  .withColumn("negative_probability", getProbability($"probability"))
  .select("tweet", "negative_probability")

predictionsDF = [tweet: string, negative_probability: double]


[tweet: string, negative_probability: double]

In [79]:
predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show(truncate=false)
}.start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@bbd0959

+--------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|tweet                                                                                                                                 |negative_probability|
+--------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|awww not going swimming.   But I have sushi.                                                                                          |0.4414830923872866  |
|@Rockergirl75 awww babe  i feel your pain                                                                                             |0.4718633713577438  |
|@TheCaptain11 Awe, I'm sorry.    LOL                                                                                                  |0.5179478977077812  |
|@MissAgnes you ok? what's causing you pain?  *hugs*

+----------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|tweet                                                                                                                                         |negative_probability|
+----------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|wow....my Berkman bobble head just doesn't seem as fun now.   What a loss.                                                                    |0.5273286976680073  |
| *cries* I'm so depressed, thanks to you my angel! &quot;I'm gonna find someone someday who might actually treat me well&quot;!!              |0.5518820155581836  |
|@myoung1989 Sounds like fun!!!! I'm on facebook, and watching national geographic.  I'm so lame, and broke as shit.  So sad.                  |0.5093944612432054  |
|@dr

+------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|tweet                                                                                                                               |negative_probability|
+------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|I havent been home since xmas break. lm a lil homesick  Faith.Hope.LOVE                                                             |0.46679644217551064 |
|is watching American dreams such a good show why did they have to cancel it                                                         |0.5398300263075214  |
|@sunshineangel89 oh me, too. Well in a few, since I still need to do something in the kitchen. Blah.                                |0.4549572539050025  |
|I hate writing up product descriptions                         

+--------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|tweet                                                                                                                                 |negative_probability|
+--------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|artsposure in Raleigh NC- today is rainy and very much many stands are already closed...                                              |0.5223410154981495  |
|I thought my boyfriend would have came to see me after work.  Looks like he's just gone home                                          |0.4391526077071773  |
|Wishing I could be at jetset with the space crew.                                                                                     |0.48659830324621894 |
|@yayKIMO who is leaving?!                          

...

+-----------------------------------------------------------------------------------------------------------------------------------+--------------------+
|tweet                                                                                                                              |negative_probability|
+-----------------------------------------------------------------------------------------------------------------------------------+--------------------+
|finals  shoot me now                                                                                                               |0.5119094203527914  |
|havent signed on to twitter in a while.  just working on the bathroon, going shopping a little later.  have a horrible headace     |0.49083956571606685 |
|OMG my itunes were sooo F*ed up took me hourssss! and my TV shows are all gone wtf?                                                |0.4924195972603118  |
|@akyna I hadn't and I'm lapping it up! Thanks.... and I hope the shar

+---------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|tweet                                                                                                                                        |negative_probability|
+---------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Quick Q: How come I have 28 followers on Twitter but I only have 12 followers on my blog?  LOL Please check out my beauty &amp; fashion blog!|0.45698314958500197 |
|Going to help a friend move his mattress and sign up for scholarship. Ugh, my lungs feel all itchy, and coughing makes my tummy hurt         |0.5063464286235158  |
|@kahlil_white me either  I think it's time for me to put on my party bloomers and do the stanky leg.                                         |0.41394852277385685 |
|@thewinch

+------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|tweet                                                                                                                                     |negative_probability|
+------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Only downside of visiting my parents, is eating ALL DAY LONG! And i've been doing so good at home. Spring training takes an L tonight.    |0.5207758140744921  |
|The sky plus box crashed yesterday I have no idea what I am going to do today my TV is gone  no 24 tonight and all my saved stuff is gone!|0.409550204656625   |
|Is missing Ana Ivanovic                                                                                                                   |0.5225633067797968  |
|Sorry for ruin gerard's par

+---------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|tweet                                                                                                                                              |negative_probability|
+---------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|I'm thinking Lion bars have declined in quality over recent years                                                                                  |0.517597819287622   |
|not enjoying revision                                                                                                                              |0.5176102312390305  |
|stupid bad dreams, I am tired but don't want to go back to bed                                                                                  

+----------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|tweet                                                                                                                                   |negative_probability|
+----------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|I don't want to go to school!  Chemistry.i hate that woman -.- Its devil inside her!                                                    |0.3839798718995863  |
|Argh.  School library has improperly configured their proxy so I cannot download an article I need. Only access I have to ACM, too      |0.4693398013336786  |
|@zackalltimelow its so sad isn't it?                                                                                                    |0.4804291227825032  |
|F**K! drop by 1  75. blogtopsites.com  